In [20]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string

In [21]:
products.head()

,category,product_id,description,size,std_sales_price,bio,basic,purchase_price
0,vegetable,v_1,Komkommer,1st,0.69,0,0,0.18
1,vegetable,v_2,Courgette,1st,0.75,0,0,0.17
2,vegetable,v_3,SnoepVegetable tomaat,500g,1.99,0,0,0.50
3,vegetable,v_4,Paprika Mix,3st,1.79,0,0,0.40
4,vegetable,v_5,Paprika,1st,0.99,0,0,0.23


In [22]:
inventory.columns[60:100]

Index(['Basis annanasstukjes', 'Biologisch wilde bosbes',
       'Biologische rozijnen', 'Biologische santana appelen',
       'Pitloze druivenmix', 'Biologische sinaasappelen',
       'Aardbeien ontkroond', 'Rundergehakt', 'Gerookte spekreepjes',
       'Spekreepjes', 'Mini gourmet hamburgers', 'Unox Gelderse rookworst',
       'Half om half gehakt', 'Rundergehakt.1', 'Slavinken',
       'Mini biefstukjes', 'Rundersoepballetjes', 'Hamburgers',
       'Unox Gelderse rookworst.1', 'Mini gourmet kipsatespiesjes',
       'Unox Magere rookworst', 'Biologische spekreepjes',
       'Sappige gelders rookworst', 'Biologisch rundergehakt',
       'Varkensbraadworst', 'Runder riblap Iers', 'rundergehakt',
       'gerookt onbijtspek', 'rundervinken', 'half of half gehakt',
       'Gekruide runderchipolata', 'Biologisch rundergehakt.1',
       'Mini gourmet slavink', 'Mini gourmet varkenshaas',
       'Zigeuner gehaktschnitzel', 'Veluwse rookworst fijn',
       'Runderbraadworst', 'Spekblokjes', '

In [23]:
product = 'Biologische spekreepjes' #select the product to analyze
transactions[transactions["description"] == product] #transactions for that product

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
1782,2/1/2018,18:44:22,101.0,24107630.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
1827,2/1/2018,18:47:38,102.0,34187107.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
2103,3/1/2018,10:47:17,117.0,1064747.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
2724,3/1/2018,19:25:33,158.0,63634047.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
4534,6/1/2018,12:45:08,274.0,43670698.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
353828,29-12-2018,20:04:32,20636.0,38621378.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354041,30-12-2018,10:08:47,20646.0,NaN,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354144,30-12-2018,12:42:05,20650.0,63634047.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354171,30-12-2018,13:15:05,20651.0,79747334.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0


In [24]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1) #convert day of year to actual timestamp

In [25]:
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)
#add nr week of year as an extra column to inventory

In [26]:
#only keep relevant columns in inventory
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
#filters promotion table to only contain discounts for chosen product
promotions_filter = promotions[promotions["description"] == product]

def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0
#adds extra column to filter inventory containing discount amount
filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


<ipython-input-26-77f34563d40f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


In [27]:
def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total
#adds total nr items in inventory for the filtered dataframe
filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

<ipython-input-27-12a88a37912f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)


In [28]:
best_before = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = np.unique(best_before)
#list of each unique best before dates
bb_dates

array([ 10,  13,  17,  20,  24,  27,  31,  34,  38,  41,  45,  48,  52,
        55,  59,  62,  66,  69,  73,  76,  80,  83,  87,  90,  94,  97,
       101, 104, 108, 111, 115, 118, 122, 125, 129, 132, 136, 139, 143,
       146, 150, 153, 157, 160, 164, 167, 171, 174, 178, 181, 185, 188,
       192, 195, 199, 202, 206, 209, 213, 216, 220, 223, 227, 230, 234,
       237, 241, 244, 248, 251, 255, 258, 262, 265, 269, 272, 276, 279,
       283, 286, 290, 293, 297, 300, 304, 307, 311, 314, 318, 321, 325,
       328, 332, 335, 339, 342, 346, 349, 353, 356, 360, 363, 367, 370])

In [29]:
df_waste = pd.DataFrame({"best before":bb_dates})
df_waste #build initial df with points where waste can occur

,best before
0,10
1,13
2,17
3,20
4,24
...,...
99,356
100,360
101,363
102,367


In [30]:
def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]

df_waste["amount"] = df_waste.apply(input_inv, axis = 1)
df_waste #amount that was received with particular expiry date

,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
...,...,...
99,356,5
100,360,5
101,363,5
102,367,5


In [31]:
df_waste = df_waste[df_waste["best before"] < 365] #only keep rows where expiry date is in same year
df_waste

,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
...,...,...
97,349,5
98,353,5
99,356,5
100,360,5


In [38]:
transactions["day"] = pd.to_datetime(transactions["day"])
#filter_transactions = transactions[transactions["description"] == product]
#converts date then filters transactions to contain selected product purchases
filtered_transactions = transactions[transactions["description"] == product]
def input_DOY(row):
    doy = pd.to_datetime(row["day"], dayfirst=True).dayofyear - 1
    return doy 
filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)
cumsum_purchases = filtered_transactions.groupby("DOY").count()["product_id"].cumsum()
#creating a dataframe,  merging it with reference so each day is accounted for
df_purchases = pd.DataFrame({"DOY":cumsum_purchases.index, "cumulative purchases":cumsum_purchases})
ref = pd.DataFrame({"doy":range(1,365)}) #reference table to match up days
joint_purchases = ref.set_index('doy').join(df_purchases)
joint_purchases.fillna(method='pad', inplace=True)#days where 0 purchases happened

first_restock = df_date[df_date["day"] == 3]["date"].values[0]
#first_restock check purchases mde before the first restock
filtered_transactions[filtered_transactions["day"] < first_restock]
#PROBLEM: IF A PRODUCT HASN'T BEEN BOUGHT IN ONE DAY IT HAS A ZERO, CAUSES AN ERROR FEW CELLS 
joint_purchases

<ipython-input-38-20a29ce21d11>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)


,DOY,cumulative purchases
doy,,
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,4.0,1.0
5,5.0,4.0
...,...,...
360,360.0,428.0
361,361.0,429.0
362,362.0,433.0


In [40]:
#cumulative sum of purchases from that product
purchases_per_day = filtered_transactions.groupby("day").count()["product_id"].cumsum()
#purchases made per day from that product
purchases_per_day_plotting = filtered_transactions.groupby("day").count()["product_id"]
#idk know what is this
purchases_per_day_plotting.index = purchases_per_day_plotting.index.dayofyear/7
purchases_per_day
#would need to generate the individual days where there are no new purchases and assign a zero value to these days to fix the cumsum()

day
2018-01-05      1
2018-01-06      4
2018-01-07      6
2018-01-09      8
2018-01-10      9
             ... 
2018-12-26    427
2018-12-27    428
2018-12-28    429
2018-12-29    433
2018-12-30    437
Name: product_id, Length: 243, dtype: int64

In [41]:
purchases_per_day.index[0].year
weeknr = []
for ind in purchases_per_day.index:
    week = datetime.date(ind.year, ind.month, ind.day).isocalendar()[1]
    weeknr.append(week)
#week number for each purchase day
purchases_per_day

day
2018-01-05      1
2018-01-06      4
2018-01-07      6
2018-01-09      8
2018-01-10      9
             ... 
2018-12-26    427
2018-12-27    428
2018-12-28    429
2018-12-29    433
2018-12-30    437
Name: product_id, Length: 243, dtype: int64

In [42]:
df_waste.drop(index=df_waste.tail(9).index, inplace=True)
#drop the last rows not to cause index issues
#STILL NEED TO BE FIXED
df_waste

C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
...,...,...
79,286,5
80,290,5
81,293,5
82,297,5


In [43]:
#df_waste["purchases"] = [purchases_per_day[i-1] for i in df_waste["best before"]]
df_waste["purchases"] = [joint_purchases['cumulative purchases'][i] for i in df_waste["best before"]]
#add cumulative sum of purchases up until that day (based on expiry dates)
df_waste

<ipython-input-43-1d0854f8769b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_waste["purchases"] = [joint_purchases['cumulative purchases'][i] for i in df_waste["best before"]]


,best before,amount,purchases
0,10,5,10.0
1,13,5,14.0
2,17,5,18.0
3,20,5,22.0
4,24,5,24.0
...,...,...,...
79,286,5,347.0
80,290,5,350.0
81,293,5,351.0
82,297,5,356.0


In [44]:
df_waste["prev purchases"] = df_waste["purchases"].diff()
df_waste #difference in purchases since a previous batch of product expired

<ipython-input-44-248a6f37d748>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_waste["prev purchases"] = df_waste["purchases"].diff()


,best before,amount,purchases,prev purchases
0,10,5,10.0,NaN
1,13,5,14.0,4.0
2,17,5,18.0,4.0
3,20,5,22.0,4.0
4,24,5,24.0,2.0
...,...,...,...,...
79,286,5,347.0,5.0
80,290,5,350.0,3.0
81,293,5,351.0,1.0
82,297,5,356.0,5.0


In [45]:
length = df_waste.shape[0]
lst_waste = [] #nr rows in waste dataframe
#amount left from the item
df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"] 
#the first row is different as there is no prev pruchases available
df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

for i in range(length):
    #iterates through the rows of the waste dataframe
    current_row = df_waste.iloc[i]
    if i == 0:
        #
        lst_waste.append(current_row["remaining"])
        continue     
    else:
        prev_row = df_waste.iloc[i-1] #previous row in dataframe
        if lst_waste[-1] < 0:
            #no idea whats happening here, I've had enough
            waste = lst_waste[-1] + current_row["remaining"]
            lst_waste.append(lst_waste[-1] + current_row["remaining"])
        else:
            lst_waste.append(current_row["remaining"])
    
df_waste["waste"] = lst_waste
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
    return converted_date
purchase_price = products[products['description'] == product]['purchase_price'].mean()
df_waste["date"] = df_waste.apply(date_converter, axis = 1)
df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
df_waste[df_waste["waste"] < 0]["waste"] = 0

df_waste['weekPrice'] = [float(products[products['description'] == product]['std_sales_price'].mean())] * len(df_waste)
#df_waste.set_index('week', inplace = True)
#print(df_waste.head())
for week_ind in list(promotions[promotions['description'] == product].index):
    week = promotions.at[week_ind, 'week']
    print(week_ind)
    # print(df_waste[['week']].dtypes)
    if week in list(np.unique(df_waste.week)):
        print(week)
        df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice'] = (1 - (promotions.at[week_ind, 'discount'] / 100)) * df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice']
        #break
df_waste['lost_Revenue'] = df_waste.waste * df_waste.weekPrice
df_waste['purc_price'] = [float(purchase_price)] * len(df_waste)
df_waste['lost_profit'] = df_waste['waste'] * (df_waste['weekPrice'] - df_waste['purc_price'])
df_waste['cost_wasted'] = df_waste['waste'] * df_waste['purc_price']
df_waste.head(50)

<ipython-input-45-080965735ec7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"]
C:\Users\zolyo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-45-080965735ec7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

,best before,amount,purchases,prev purchases,remaining,waste,date,week,weekPrice,lost_Revenue,purc_price,lost_profit,cost_wasted
0,10,5,10.0,NaN,-5.0,-5.0,2018-01-11,2,2.69,-13.45,0.61,-10.40,-3.05
1,13,5,14.0,4.0,1.0,-4.0,2018-01-14,2,2.69,-10.76,0.61,-8.32,-2.44
2,17,5,18.0,4.0,1.0,-3.0,2018-01-18,3,2.69,-8.07,0.61,-6.24,-1.83
3,20,5,22.0,4.0,1.0,-2.0,2018-01-21,3,2.69,-5.38,0.61,-4.16,-1.22
4,24,5,24.0,2.0,3.0,1.0,2018-01-25,4,2.69,2.69,0.61,2.08,0.61
5,27,5,29.0,5.0,0.0,0.0,2018-01-28,4,2.69,0.00,0.61,0.00,0.00
6,31,5,33.0,4.0,1.0,1.0,2018-02-01,5,2.69,2.69,0.61,2.08,0.61
7,34,5,35.0,2.0,3.0,3.0,2018-02-04,5,2.69,8.07,0.61,6.24,1.83
8,38,5,42.0,7.0,-2.0,-2.0,2018-02-08,6,2.69,-5.38,0.61,-4.16,-1.22
9,41,5,46.0,4.0,1.0,-1.0,2018-02-11,6,2.69,-2.69,0.61,-2.08,-0.61


In [46]:
#promotions[(promotions['description'] == product) & (promotions['week'] == week)]['discount']
#promotions[promotions['description'] == product]['week']
promotions[(promotions['description'] == product) ]

,week,category,product_id,description,discount


In [47]:
print(products[products['description'] == 'Half om half gehakt'])

   category product_id          description  size  std_sales_price  bio  \
70     meat        m_6  Half om half gehakt  500g             2.69    0   

    basic  purchase_price  
70      0            0.55  


In [48]:
df_waste[df_waste['waste']>0][['waste', 'lost_Revenue', 'lost_profit', 'cost_wasted']].sum()

waste            60.0
lost_Revenue    161.4
lost_profit     124.8
cost_wasted      36.6
dtype: float64

In [49]:
print('The number of products wasted in the dataset for {} is {} items'.format(product, df_waste[df_waste['waste']>0]['waste'].sum()))

The number of products wasted in the dataset for Biologische spekreepjes is 60.0 items


In [50]:
df_waste.loc[:,:].where(df_waste.loc[:,'waste']>0)

,best before,amount,purchases,prev purchases,remaining,waste,date,week,weekPrice,lost_Revenue,purc_price,lost_profit,cost_wasted
0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,24.0,5.0,24.0,2.0,3.0,1.0,2018-01-25,4.0,2.69,2.69,0.61,2.08,0.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
81,293.0,5.0,351.0,1.0,4.0,2.0,2018-10-21,42.0,2.69,5.38,0.61,4.16,1.22
82,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df_waste[df_waste['waste']>0]['waste'].sum()

60.0

In [52]:
transactions[transactions['description'] == 'Biologische prei'].groupby('day').count().iloc[0:14, :]

,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
day,,,,,,,,,,,
2018-01-01,2,2,1,2,2,2,2,2,2,2,2
2018-01-02,1,1,0,1,1,1,1,1,1,1,1
2018-01-03,3,3,3,3,3,3,3,3,3,3,3
2018-01-04,9,9,9,9,9,9,9,9,9,9,9
2018-01-05,4,4,4,4,4,4,4,4,4,4,4
2018-01-06,3,3,1,3,3,3,3,3,3,3,3
2018-01-07,7,7,4,7,7,7,7,7,7,7,7
2018-01-08,5,5,5,5,5,5,5,5,5,5,5
2018-01-09,5,5,5,5,5,5,5,5,5,5,5


In [53]:
df_waste["waste nn"] = [i if i > 0 else 0 for i in df_waste["waste"]]
#takes nonnegative values which is counted as waste
waste_group = df_waste.groupby("week").sum()
waste_group["weekly purchases"] = waste_group["purchases"].diff()


<ipython-input-53-08904a856e7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_waste["waste nn"] = [i if i > 0 else 0 for i in df_waste["waste"]]


## `df_waste` is prepared for plotting

In [54]:
waste_group

,best before,amount,purchases,prev purchases,remaining,waste,weekPrice,lost_Revenue,purc_price,lost_profit,cost_wasted,waste nn,weekly purchases
week,,,,,,,,,,,,,
2,23,10,24.0,4.0,-4.0,-9.0,5.38,-24.21,1.22,-18.72,-5.49,0.0,NaN
3,37,10,40.0,8.0,2.0,-5.0,5.38,-13.45,1.22,-10.40,-3.05,0.0,16.0
4,51,10,53.0,7.0,3.0,1.0,5.38,2.69,1.22,2.08,0.61,1.0,13.0
5,65,10,68.0,6.0,4.0,4.0,5.38,10.76,1.22,8.32,2.44,4.0,15.0
6,79,10,88.0,11.0,-1.0,-3.0,5.38,-8.07,1.22,-6.24,-1.83,0.0,20.0
7,93,10,102.0,7.0,3.0,2.0,5.38,5.38,1.22,4.16,1.22,2.0,14.0
8,107,10,114.0,5.0,5.0,5.0,5.38,13.45,1.22,10.40,3.05,5.0,12.0
9,121,10,134.0,13.0,-3.0,-3.0,5.38,-8.07,1.22,-6.24,-1.83,0.0,20.0
10,135,10,166.0,17.0,-7.0,-15.0,5.38,-40.35,1.22,-31.20,-9.15,0.0,32.0


In [55]:
waste_group["week"] = waste_group.index

In [58]:
import plotly.express as px
fig = px.line(waste_group, x = "week", y = "waste nn", title="Absolute waste of {} per week".format(product))
fig.add_scatter(x=waste_group["week"], y=waste_group["amount"], mode="lines")
fig.add_scatter(x=waste_group["week"], y=waste_group["weekly purchases"], mode="lines")
fig.update_yaxes(title="Waste amount")
fig.show()

In [112]:
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [71]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]

In [86]:
daydate = df_date[df_date["day"] == 2]["date"].values[0]

filter_transactions[filter_transactions["day"] <= daydate]


,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
89,2018-01-01,10:40:13,6.0,54211938.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
467,2018-01-01,14:21:35,26.0,51433968.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
573,2018-01-01,15:36:46,33.0,30794082.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
700,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
808,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
821,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
824,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
879,2018-01-01,18:46:48,49.0,15382238.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29093,2018-01-02,12:52:06,1714.0,28719528.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29134,2018-01-02,13:44:14,1718.0,39309224.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0


In [111]:
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [52]:
daydate

numpy.datetime64('2018-01-15T00:00:00.000000000')

In [15]:
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})